In [13]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import squareform, pdist
from scipy.stats import zscore
from sklearn import linear_model
from math import cos, sin, pi, isnan
import xlsxwriter
import nibabel as nib
from nilearn.glm.second_level import SecondLevelModel
from nilearn.image import load_img, threshold_img
from nilearn.glm import threshold_stats_img
from nilearn import plotting
import matplotlib.pyplot as plt
from nltools import Brain_Data

In [14]:
topdir='/home/haiyanwu/nas_data/experiment_data/uarep_only/'
glmdir=topdir+'1.GLM/results/GLM6uarep_combination/betamap/'
maskdir=topdir+'3.ROIextraction/masks/'

In [15]:
subList=[101,102,103,105,106,107,108,109,110,111,112,115,116,117,118,119,120,121,123,125,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143]
len(subList)

37

# RDM regression within ROIs

# matrix to compute dissimilarities
### islie, consisitency, money

In [16]:

all=pd.read_csv('/home/haiyanwu/nas_data/experiment_data/uarep_only/2.uarep-hddm/fMRI_sessionData_DDM_rep.csv')
all=all[all['RT']<=4]



metric_model='euclidean'
for subno in subList:
    dist=[]

    data=all.loc[all['subj']==subno,:]
    data=data.groupby(['former_diff','diff','islie']).size().reset_index()
    data=data[data['former_diff']!=0]
    data.reset_index(inplace=True)



    dist1=pdist(data.loc[:,['islie']],metric=metric_model)
    dist1[dist1==1]=1/np.sum(dist1)
    dist1[dist1==0]=-1/len(dist1[dist1==0])


    data['con']=1
    data.loc[data['former_diff']<0,'con']=0
 

    dist2=pdist(data.loc[:,['con']],metric=metric_model)
    dist2[dist2==1]=1/np.sum(dist2)
    dist2[dist2==0]=-1/len(dist2[dist2==0])


    data['mon']=1
    data.loc[data['diff']<0,'mon']=0
 

    dist3=pdist(data.loc[:,['mon']],metric=metric_model)
    dist3[dist3==1]=1/np.sum(dist3)
    dist3[dist3==0]=-1/len(dist3[dist3==0])


   



    dist.append(dist1)
    dist.append(dist2)
    dist.append(dist3)

  

    subdata=pd.DataFrame(dist)
   
    subdata.to_csv('sepMatrix/sub-%03d.csv'%subno)

  

# RDM regression within ROIs

In [17]:
ROIs=['BA9-dlPFC','BA24-dACC','BA34-hip','lDLPFC','rDLPFC','ventral_striatum','putamen','rTPJ','lTPJ','BA21-MTG','BA23-PCC','BA47-vlPFC','mPFC',
        'NAcc','ACC','BA33-rACC','amyg','amygL','amygR','precuneus','caudate','BA45-IFG','BA10-FPC']
len(ROIs)

35

In [18]:
def drop_na(z):
    ztt=[]
    for i in z:
        if not isnan(i):
            ztt.append(i)
    return ztt

In [19]:
all=pd.read_csv('/home/haiyanwu/nas_data/experiment_data/uarep_only/2.uarep-hddm/fMRI_sessionData_DDM_rep.csv')
all=all[all['RT']<=4]

results={}
writer=pd.ExcelWriter('separation.xlsx',engine='xlsxwriter')
for roi in ROIs:
    results[roi]=[]
for subno in subList:

    modelRDM=pd.read_csv('sepMatrix/sub-%03d.csv'%subno,index_col=0)


    

    islie=drop_na(list(modelRDM.iloc[0,:]))
    con=drop_na(list(modelRDM.iloc[1,:]))
    mon=drop_na(list(modelRDM.iloc[2,:]))
    
            
    
    d=all[all['subj']==subno]
    d=d[d['RT']<=4]
    d=d.groupby(['former_diff','diff','islie']).size().reset_index()
    d=d[d['former_diff']!=0]
    d['combination']=list(map(lambda x,y,z:'con%d_mon%d_islie%d'%(x,y,z),d['former_diff'],d['diff'],d['islie']))



    betas=Brain_Data()
    

    for cname in d['combination'].to_list():
        fname=glmdir+'sub-%03d/sub-%03d_%s_eff_map.nii.gz'%(subno,subno,cname)
        betas=betas.append(Brain_Data(fname))

    for roi in ROIs:

        mask=Brain_Data(maskdir+'Mask_%s.nii'%(roi))
        brainDist = betas.apply_mask(mask).distance(metric='correlation')
    

       
        X=pd.DataFrame(brainDist.data)
        M=pd.DataFrame([islie,con,mon])#
        results[roi].append((X.T@M.T).iloc[0,:].to_list())
      
        
for roi in ROIs:
    results[roi]=pd.DataFrame(results[roi])
    results[roi].to_excel(writer,sheet_name='%s'%roi,startcol=0,engine='xlsxwriter')
writer.save()
    
    
    





/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/lib/python3.9/site-packages/nilearn/image/resampling.py:273: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/home/haiyanwu/miniconda3/li